`label transfer` & overlapping analysis of acute-covid & covid-flu datasets

Adapted from Can's codes

In [3]:
rm(list = ls())
getwd()

[1] "/gpfs/gibbs/project/tsang/ah2426/sc_annotation/devel_ao"

In [4]:
### load libraries

library(Seurat)
library(matrixStats)
library(plyr)
library(tidyverse)
library(future)
library(future.apply)
plan("multisession", workers = 4)
options(future.globals.maxSize = 20000 * 1024^2)
library(pheatmap)

In [38]:
### load data

PATH_ACUTE="/gpfs/gibbs/pi/csei/users/CanLiu/projects/pub_datasets/Brescia_covid/brescia_paper1_seurat.rds"
PATH_COVFL="/gpfs/gibbs/pi/csei/users/CanLiu/projects/pub_datasets/covid_flu/GSE206265_covid_flu.CITEseq.Seurat.obj.RDS"

acute_covid = readRDS(PATH_ACUTE) # data are under limmaCITE are batch-normalized
covid_flu   = readRDS(PATH_COVFL)

In [39]:
### trim data

# save spaces
DefaultAssay(covid_flu) = 'CITE'
covid_flu@assays$HTO = NULL
covid_flu@assays$SCT = NULL
covid_flu@assays$RNA = NULL

DefaultAssay(acute_covid) = 'limmaCITE'
acute_covid@assays$HTO = NULL
acute_covid@assays$RNA = NULL

# rm dbl & unknown celltypes in acute_covid
rm_celltype = unique(acute_covid$WCTmergedcelltype)[str_detect(unique(acute_covid$WCTmergedcelltype), pattern = "blt|Unknown|gated")]
print(rm_celltype)

acute_covid = subset(acute_covid, subset=WCTmergedcelltype %in% rm_celltype, invert=TRUE)

cat('\n-- acute_covid info --\n')
acute_covid

cat('\n-- covid_flu info --\n')
covid_flu

 [1] "gated_out"                  "Unknown_Unknown"           
 [3] "Dblt_B.Tcell.dblt"          "Dblt"                      
 [5] "Mono_Classical_dblt.Mono.T" "Dblt_B.T"                  
 [7] "Unknown"                    "Dblt_T.DC.dblt"            
 [9] "B_Naive_B.Mono.dblt"        "B_Mem_B.T.dblt"            
[11] "Dblt_B.Gran.dblt"           "Dblt_B.T.dblt"             
[13] "Dblt_B.Mono.dblt"          

-- acute_covid info --


An object of class Seurat 
384 features across 397468 samples within 2 assays 
Active assay: limmaCITE (192 features, 0 variable features)
 1 other assay present: CITE


-- covid_flu info --


An object of class Seurat 
138 features across 632100 samples within 1 assay 
Active assay: CITE (138 features, 0 variable features)
 1 dimensional reduction calculated: adt.umap

### Consistency between two datasets

In [34]:
### feature consistency -- there must be a better way to have simpler codes

# cite_assay  = GetAssayData(acute_covid, assay = 'CITE', slot = 'data')
# limma_assay = GetAssayData(acute_covid, assay = 'limmaCITE', slot = 'data')

# # add prefix
# rownames(cite_assay) = paste0('PROT-', rownames(cite_assay))
# rownames(limma_assay) = paste0('PROT-', rownames(limma_assay))

# # rename some features
# rownames(cite_assay)[rownames(cite_assay) == "PROT-TCRVd2"] <- "PROT-TCRVdelta2"
# rownames(cite_assay)[rownames(cite_assay) == "PROT-TCRVa7.2"] <- "PROT-TCRValpha7p2"
# rownames(limma_assay)[rownames(limma_assay) == "PROT-TCRVd2"] <- "PROT-TCRVdelta2"
# rownames(limma_assay)[rownames(limma_assay) == "PROT-TCRVa7.2"] <- "PROT-TCRValpha7p2" 

# # extract overlapping features with covid_flu
# DefaultAssay(covid_flu) = 'CITE'
# cite_assay = cite_assay[rownames(cite_assay) %in% rownames(covid_flu), ]
# limma_assay = limma_assay[rownames(limma_assay) %in% rownames(covid_flu), ]

# # plug in assay with updated rownames
# acute_covid[["CITE"]] = CreateAssayObject(data = cite_assay)
# acute_covid[["limmaCITE"]] = CreateAssayObject(data = limma_assay)

# match for covid_flu as well
# cite_assay = GetAssayData(covid_flu, assay = 'CITE', slot = 'data')
# cite_assay = cite_assay[rownames(cite_assay) %in% rownames(acute_covid), ]
# acute_covid[["CITE"]] = CreateAssayObject(data = cite_assay)

# cat('\n-- acute_covid info --\n')
# acute_covid

# cat('\n-- covid_flu info --\n')
# covid_flu

# cat('\n -- Overlaped features --\n')
# rownames(acute_covid)

In [41]:
# rename some of the features
acute_covid = rename_features(acute_covid, old='TCRVd2', new='TCRVdelta2', assay='CITE')
acute_covid = rename_features(acute_covid, old='TCRVa7.2', new='TCRValpha7p2', assay='CITE')
acute_covid = rename_features(acute_covid, old='TCRVd2', new='TCRVdelta2', assay='limmaCITE')
acute_covid = rename_features(acute_covid, old='TCRVa7.2', new='TCRValpha7p2', assay='limmaCITE')

# add prefix
acute_covid = add_prefix(acute_covid, prefix = 'PROT-', assay = 'CITE')
acute_covid = add_prefix(acute_covid, prefix = 'PROT-', assay = 'limmaCITE')

# extract overlapping features w/ covid_flu
acute_covid = match_features(acute_covid, covid_flu, seurat1_assay = 'CITE')
acute_covid = match_features(acute_covid, covid_flu, seurat1_assay = 'limmaCITE')

# match for covid_flu too
covid_flu = match_features(covid_flu, acute_covid, seurat1_assay = 'CITE')

cat('\n-- acute_covid info --\n')
acute_covid

cat('\n-- covid_flu info --\n')
covid_flu

cat('\n -- Overlaped features --\n')
rownames(acute_covid)

# it'd seem that overlapped features don't have isotypes!


-- acute_covid info --


An object of class Seurat 
230 features across 397468 samples within 2 assays 
Active assay: limmaCITE (115 features, 0 variable features)
 1 other assay present: CITE


-- covid_flu info --


An object of class Seurat 
115 features across 632100 samples within 1 assay 
Active assay: CITE (115 features, 0 variable features)
 1 dimensional reduction calculated: adt.umap


 -- Overlaped features --


[1] "PROT-CD86"         "PROT-CD274"        "PROT-CD11b"       
  [4] "PROT-CD40"         "PROT-CD3"          "PROT-CD8"         
  [7] "PROT-CD56"         "PROT-CD19"         "PROT-CD33"        
 [10] "PROT-CD11c"        "PROT-HLA-ABC"      "PROT-CD45RA"      
 [13] "PROT-CD123"        "PROT-CD7"          "PROT-CD194"       
 [16] "PROT-CD4"          "PROT-CD14"         "PROT-CD16"        
 [19] "PROT-CD25"         "PROT-CD45RO"       "PROT-CD279"       
 [22] "PROT-CD20"         "PROT-CD31"         "PROT-IgM"         
 [25] "PROT-CD5"          "PROT-CD183"        "PROT-CD195"       
 [28] "PROT-CD32"         "PROT-CD196"        "PROT-CD185"       
 [31] "PROT-CD103"        "PROT-CD69"         "PROT-CD62L"       
 [34] "PROT-CD161"        "PROT-CD152"        "PROT-CD223"       
 [37] "PROT-KLRG1"        "PROT-CD27"         "PROT-CD95"        
 [40] "PROT-HLA-DR"       "PROT-CD1c"         "PROT-CD64"        
 [43] "PROT-CD141"        "PROT-CD1d"         "PROT-CD314"       
 [46] "PROT-CD57"         "PROT-CD278"        "PROT-CD39"        
 [49] "PROT-CX3CR1"       "PROT-CD24"         "PROT-CD21"        
 [52] "PROT-CD244"        "PROT-CD137"        "PROT-CD163"       
 [55] "PROT-CD2"          "PROT-CD303"        "PROT-IgD"         
 [58] "PROT-CD18"         "PROT-CD28"         "PROT-CD38"        
 [61] "PROT-CD127"        "PROT-CD71"         "PROT-CD155"       
 [64] "PROT-CD112"        "PROT-CD47"         "PROT-CD154"       
 [67] "PROT-CD52"         "PROT-CD49f"        "PROT-CD44"        
 [70] "PROT-TIGIT"        "PROT-CD335"        "PROT-CD146"       
 [73] "PROT-CD107a"       "PROT-CD35"         "PROT-CD272"       
 [76] "PROT-CD58"         "PROT-CD11a"        "PROT-CD79b"       
 [79] "PROT-CD169"        "PROT-CD268"        "PROT-CD54"        
 [82] "PROT-CD62P"        "PROT-TCRab"        "PROT-CD122"       
 [85] "PROT-CD267"        "PROT-CD41"         "PROT-CD83"        
 [88] "PROT-CD124"        "PROT-CD226"        "PROT-CD29"        
 [91] "PROT-CD49b"        "PROT-CD81"         "PROT-CD45"        
 [94] "PROT-CD22"         "PROT-CD26"         "PROT-CD36"        
 [97] "PROT-CD158"        "PROT-CD49d"        "PROT-CD73"        
[100] "PROT-TCRValpha7p2" "PROT-TCRVdelta2"   "PROT-LOX-1"       
[103] "PROT-CD158b"       "PROT-CD158e1"      "PROT-CD319"       
[106] "PROT-CD99"         "PROT-CLEC12A"      "PROT-CD94"        
[109] "PROT-CD85j"        "PROT-CD23"         "PROT-CD328"       
[112] "PROT-CD82"         "PROT-CD101"        "PROT-CD88"        
[115] "PROT-CD224"

In [ ]:
### celltype label consistency -- there's a transfer table created by Can...


In [42]:
### adjust to right formats for RF training (for data_preprocess.R)

# rename celltype query
covid_flu$cell.type = covid_flu$cell.type
covid_flu$coarse.cell.type = covid_flu$coarse.cell.type

acute_covid$cell.type = acute_covid$WCTmergedcelltype
acute_covid$coarse.cell.type = acute_covid$coursecelltype

### Label transfer

Ideally, isotype ctrl features (e.g. `ArmenianHamsterIgGiso`) should be removed since they are not informative. 

Here, try top PCs first as in Can's analysis for simplicity.

In [43]:
# follow Can's pipeline

# DefaultAssay(acute_covid) = 'limmaCITE'
# DefaultAssay(covid_flu) = 'CITE'

# covid_list <- list("acute_covid" = acute_covid, "covid_flu" = covid_flu)
# covid_list <- lapply(X = covid_list, FUN = function(x) {
#                         x = FindVariableFeatures(x, nfeatures = 100) })
# features = SelectIntegrationFeatures(object.list = covid_list)

# covid_list <- lapply(X = covid_list, FUN = function(x) {
#     x <- ScaleData(x, features = features)
#     x <- RunPCA(x, features = features, verbose=FALSE)
# })

# ElbowPlot(covid_list$covid_flu)


# start label transfer

# To save time, just use the label transfer results computed by Can

PRECOMPUTE = '/gpfs/gibbs/project/tsang/cl2626/ondemand_data/cell_ann_stat'

predictions = readRDS(paste0(PRECOMPUTE, '/labeltransfer_predictions.rds'))
dim(predictions)
head(predictions)

[1] 632100    100

,predicted.id,prediction.score.Mono_NonClassical,prediction.score.B_Naive_IgMpos,prediction.score.Mono_Classical,prediction.score.NK_CD16hi,prediction.score.B_Mem_IgMneg,prediction.score.gammadeltaT,prediction.score.Mono_Classical_CD163hi,prediction.score.Mono_Classical_CD71pos,prediction.score.B_Mem_CD11cpos,⋯,prediction.score.Mono_Intermediate_clump,prediction.score.CD4_Mem_CD41hi,prediction.score.Mono_Classical_CD1dpos,prediction.score.Mono_NonClassical_IgPos,prediction.score.Mono_Intermediate,prediction.score.CD4_Mem_MAIT,prediction.score.CD4_Mem_CD22hi,prediction.score.cDC_CD141pos,prediction.score.CD4_Mem_CD69pos,prediction.score.max
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Batch1_COVFLU_CITE_multi5P12_ACTGATGGTTCAGCGC-1,Mono_Classical,0.01387509,0.000000,0.9141866,0,0,0,0.02812370,0,0.000000000,⋯,0,0,0,0,0,0,0,0,0,0.9141866
Batch1_COVFLU_CITE_multi5P16_CCCAGTTTCGGCTTGG-1,B_Naive_IgMpos,0.00000000,0.552819,0.0000000,0,0,0,0.00000000,0,0.001842123,⋯,0,0,0,0,0,0,0,0,0,0.5528190
Batch1_COVFLU_CITE_multi5P09_GCATGCGCAGCCTATA-1,CD4_Naive,0.00000000,0.000000,0.0000000,0,0,0,0.00000000,0,0.000000000,⋯,0,0,0,0,0,0,0,0,0,0.6672915
Batch1_COVFLU_CITE_multi5P11_CATCCACAGCTCTCGG-1,Mono_Classical,0.00000000,0.000000,0.9527907,0,0,0,0.01811537,0,0.000000000,⋯,0,0,0,0,0,0,0,0,0,0.9527907
Batch1_COVFLU_CITE_multi5P02_TTGTAGGCAGTATCTG-1,CD4_Mem_CM,0.00000000,0.000000,0.0000000,0,0,0,0.00000000,0,0.000000000,⋯,0,0,0,0,0,0,0,0,0,0.5417251
Batch1_COVFLU_CITE_multi5P02_GATGCTACAGCTTAAC-1,CD4_Naive,0.00000000,0.000000,0.0000000,0,0,0,0.00000000,0,0.000000000,⋯,0,0,0,0,0,0,0,0,0,0.8819621


### Output data

In [46]:
table(covid_flu$Batch)
table(acute_covid$batch)


Batch1 Batch2 Batch3 
204881 222475 204744 


Batch1 Batch2 Batch3 
 29375 114223 134815 

#### Scenario 1: predict across datasets 

1. From acute-covid --> covid-flu (use only 1 batch)
2. From acute-covid --> covid-flu (use all cells)

In [50]:
# save for all cells

# acute_covid2 contains only the limmaCITE assay
acute_covid2 = acute_covid
DefaultAssay(acute_covid2) = 'limmaCITE'
acute_covid2@assays$CITE = NULL
acute_covid2 = RenameAssays(acute_covid2, limmaCITE='CITE')
acute_covid2

# acute_covid contains only the CITE assay
DefaultAssay(acute_covid) = 'CITE'
acute_covid2@assays$limmaCITE = NULL

# save them all
saveRDS(covid_flu,    "../data/covid_flu.cite.allcells.dsb.RDS")
saveRDS(acute_covid,  "../data/acute_cov.cite.allcells.dsb.RDS")
saveRDS(acute_covid2, "../data/acute_cov.limmaCite.allcells.dsb.RDS")

Renaming default assay from limmaCITE to CITE

Warning message:
“Cannot add objects with duplicate keys (offending key: limmacite_) setting key to original value 'cite_'”


An object of class Seurat 
115 features across 397468 samples within 1 assay 
Active assay: CITE (115 features, 0 variable features)

#### Scenario 2: predict across batches

1. Within covid-flu (first)
2. Within acute-covid (use the CITE array, not limma CITE)


In [ ]:
for (bth in c('Batch1', 'Batch2', 'Batch3')) {

    covid_flu_b = subset(covid_flu, subset = Batch == bth)
    acute_cov_b = subset(acute_covid, subset = batch == bth) # save only the cite array

    saveRDS(covid_flu_b, paste0("../data/covid_flu.cite.", bth, ".dsb.RDS"))
    saveRDS(acute_cov_b, paste0("../data/acute_cov.cite.", bth, ".dsb.RDS"))
    
}

#### Scenario 3: include label transfer information

--> DONE label transfer as above

For faster training, use 1 batch first

### Investigation of classes

Check class label numbers and think about which labels to include

In [5]:
### load simpler data version 

acute_cov = readRDS("../data/acute_cov.cite.allcells.dsb.RDS")
covid_flu = readRDS("../data/covid_flu.cite.allcells.dsb.RDS")

In [8]:
# coarse-grained class

table(as.character(acute_cov$coarse.cell.type))
table(as.character(covid_flu$coarse.cell.type))

# coarse-grained by batch
table(as.character(covid_flu$coarse.cell.type),
      covid_flu$Batch)


              B             CD4             CD8             dim             DNT 
          42006          141077           62759             396            5299 
    gammadeltaT    Granulocytes            MAIT            Mono              NK 
           5925             826            6379           88647           34870 
             PB         pDC.cDC       Platelets             RBC           Tcell 
           2113            1887            3573             232              35 
TCRVbeta13.1pos   TissueResMemT 
             87            1357 


          B         CD4         CD8         cDC     gdT-Vd2        HSPC 
      42962      208129       95197        7505       14904         734 
        ILC Mac-or-Mono        MAIT        Mono Mono-T-dblt        Neut 
        502        2664       18509      147698        2502        1010 
         NK         pDC Plasmablast    Platelet 
      80552        3859         670        4703 

             
              Batch1 Batch2 Batch3
  B            10597  18691  13674
  CD4          68247  68484  71398
  CD8          33089  34175  27933
  cDC           2130   2893   2482
  gdT-Vd2       4617   4789   5498
  HSPC           253    257    224
  ILC            186    196    120
  Mac-or-Mono   1920    421    323
  MAIT          6050   6779   5680
  Mono         44410  54331  48957
  Mono-T-dblt   1199    666    637
  Neut           291    391    328
  NK           29196  27067  24289
  pDC           1275   1378   1206
  Plasmablast    150    312    208
  Platelet      1271   1645   1787

In [10]:
# fine-grained class

table(as.character(acute_cov$cell.type))
table(as.character(covid_flu$cell.type))

# coarse-grained by batch
table(as.character(covid_flu$cell.type),
      covid_flu$Batch)


                  B_Mem_CD11cpos                     B_Mem_CD31hi 
                            1949                               16 
                   B_Mem_CD62Lhi                    B_Mem_CD69pos 
                             548                               91 
                   B_Mem_CD71pos                 B_Mem_IgG.IgApos 
                             967                              185 
            B_Mem_IgG.IsoBinding                     B_Mem_IgMneg 
                             441                            11099 
                    B_Mem_IgMpos                     B_Mem_IgMvar 
                            4973                             1594 
                B_Naive_CD11cpos                   B_Naive_CD38hi 
                             437                              366 
                 B_Naive_CD69pos           B_Naive_IgG.IsoBinding 
                              41                              397 
             B_Naive_IgMlo.IgDhi                   B_Naive_Ig


               B_Mem              B_Naive B_Naive_Intermediate 
               11516                22749                 8697 
              CD4_CM               CD4_EM            CD4_Naive 
               57898                23936               110879 
   CD4_platelet_bind              CD4_Tfh             CD4_Treg 
                5952                 1302                 8162 
              CD8_CM               CD8_EM            CD8_Naive 
               10732                33099                31710 
   CD8_proliferating            CD8_TEMRA              CD8_TRM 
                 812                16101                 2743 
                 cDC              gdT-Vd2                 HSPC 
                7505                14904                  734 
                 ILC          Mac-or-Mono                 MAIT 
                 502                 2664                18509 
      Mono_Classical    Mono_Intermediate    Mono_NonClassical 
              109163                181

                      
                       Batch1 Batch2 Batch3
  B_Mem                  2427   4955   4134
  B_Naive                6933   9331   6485
  B_Naive_Intermediate   1237   4405   3055
  CD4_CM                16565  20007  21326
  CD4_EM                10733   7347   5856
  CD4_Naive             36611  35914  38354
  CD4_platelet_bind      1865   1821   2266
  CD4_Tfh                 424    486    392
  CD4_Treg               2049   2909   3204
  CD8_CM                 3329   4072   3331
  CD8_EM                12966  12920   7213
  CD8_Naive              9523  10085  12102
  CD8_proliferating       234    292    286
  CD8_TEMRA              6155   5862   4084
  CD8_TRM                 882    944    917
  cDC                    2130   2893   2482
  gdT-Vd2                4617   4789   5498
  HSPC                    253    257    224
  ILC                     186    196    120
  Mac-or-Mono            1920    421    323
  MAIT                   6050   6779   5680
  Mono_Cl

### helper functions

In [40]:
rename_features = function (seurat_obj, old, new, assay) {
    # Returns a seurat object where the feature names are substituted from
    # old to new
    # 
    # Parameter dataframe: a data.frame with rownames
    # 
    # Parameter old: an old feature to be substituted
    # Preconditions: a character
    # 
    # Parameter new: a new feature to substitute the old feature
    # Preconditions: a character

    tmp = GetAssayData(seurat_obj, assay=assay, slot='data')

    rownames(tmp)[rownames(tmp) == old] <- new
    seurat_obj[[assay]] = CreateAssayObject(data = tmp)

    return(seurat_obj)

}


add_prefix = function (seurat_obj, prefix, assay = 'CITE') {

    tmp = GetAssayData(seurat_obj, assay=assay, slot='data')
    rownames(tmp) = paste0(prefix, rownames(tmp))

    seurat_obj[[assay]] = CreateAssayObject(data = tmp)

    return(seurat_obj)
    
}


match_features = function (seurat1, seurat2, seurat1_assay='CITE') {
    # Returns a seurat object with the same feature labels as seurat2
    # 
    # Noneoverlapped features in seurat1 would be removed; the assay in seurat2
    # that seurat1 is matching is 'CITE'
    # 
    # Parameter seurat1 & seurat2: seurat objects
    # Preconditions: features in seurat1 & seurat2 have overlaps, features in
    # seurat2 should be a subset of features in seurat1
    # 
    # Parameter seurat1_assay: an assay within seurat1
    # Preconditions: choose from 'limmaCITE' or 'CITE' as in the case of 
    # acute_covid

    tmp = GetAssayData(seurat1, assay=seurat1_assay, slot='data')
    
    tmp = tmp[rownames(tmp) %in% rownames(seurat2), ]
    seurat1[[seurat1_assay]] = CreateAssayObject(data = tmp)    
    
    return(seurat1)
    
}

In [ ]:
# manipulate_rownames = function (dataframe, FUNC) {

#     # get data from a seurat assay
#     data = GetAssayData(seurat_obj, assay = assay, slot = 'data')

#     # update the rownames according to FUNC
#     rowname = rownames(data)
#     new_rowname = FUNC(array=rowname, prefix='PROT-')
#     rownames(data) = new_rowname

#     # put back the data assay with rownames changed
#     seurat_obj[[assay]] = CreateAssayObject(data = data)
#     return(seurat_obj)
    
# }

# add_prefix = function (array=NULL, prefix) {
#     return(paste0(prefix, array))
# }

# test = manipulate_rownames( acute_covid, assay = 'limmaCITE', FUNC = add_prefix )
# test